In [2]:
import numpy as np

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import kernels

In [3]:
import solution
from solution import Model
from solution import cv_eval


IndentationError: expected an indented block (solution.py, line 178)

In [ ]:
#copy code to override default classes

In [ ]:
#def main():
train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')

# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

In [ ]:
### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
kernel = kernels.sklearn_best()
K_cv = 10
cv_m = cv_eval(cv_splits = K_cv, kernel = kernel, 
               use_skit_learn = True, public_score_run=False) # Choose whether to use custom model or scikit learn
val_cost_array = cv_m.run_cross_validation(train_x, train_y)

print(np.mean(val_cost_array))
print(np.std(val_cost_array))

In [1]:
a = kernels.rbf_kernel(train_x,train_x)

NameError: name 'kernels' is not defined